# 快速应用构建与 API 部署教程 (FastAPI, Gradio, Streamlit)

欢迎来到快速应用构建与 API 部署教程！训练好机器学习模型后，下一步往往是将其部署为可供调用的 API 服务，或者创建一个交互式的 Web 应用/Demo 来展示其功能。

本教程将独立地介绍三个流行的 Python 库，它们可以帮助你快速完成这些任务：

1.  **FastAPI**: 一个现代、高性能的 Web 框架，特别适合构建 API。
2.  **Gradio**: 一个非常易于使用的库，专门用于快速为机器学习模型创建可定制的 Web UI Demo。
3.  **Streamlit**: 一个用于创建美观的数据科学 Web 应用的开源框架，使用纯 Python 脚本。

我们将通过部署一个简单的 Iris 分类模型，分别展示每个库的基础用法。

**本教程结构：**
1.  准备工作（安装库、训练并保存示例模型）。
2.  使用 FastAPI 构建 API。
3.  使用 Gradio 创建交互式 Demo。
4.  使用 Streamlit 构建数据应用。
5.  比较与总结。

## 1. 准备工作

安装必要的库，并训练、保存一个简单的模型供后续部分使用。

### 1.1 安装库

```bash
pip install fastapi "uvicorn[standard]" pydantic 
pip install gradio
pip install streamlit

# 其他依赖
pip install scikit-learn joblib numpy pandas
```
**注意**: FastAPI 和 Streamlit 应用通常需要在终端中启动。

In [ ]:
# --- 公共依赖导入 (仅用于模型训练) ---
import joblib
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
import os

print("Preparing a simple model...")
# --- 训练并保存一个简单的示例模型 --- 
model_filename = 'simple_iris_model_deploy.joblib'
iris_data = load_iris()
X, y = iris_data.data, iris_data.target
target_names = iris_data.target_names

simple_model = LogisticRegression(max_iter=200)
simple_model.fit(X, y)

joblib.dump(simple_model, model_filename)
print(f"Simple Iris classification model trained and saved to {model_filename}")
print(f"Target names: {target_names}")

## 2. 使用 FastAPI 构建 API

FastAPI 利用 Python 类型提示来定义 API 的输入输出，自动进行数据验证并生成交互式文档。

In [ ]:
%%writefile main_fastapi_deploy.py
# --- FastAPI: 文件内容 ---
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import joblib
import numpy as np
import os

# 配置
MODEL_FILENAME = 'simple_iris_model_deploy.joblib'
IRIS_TARGET_NAMES = ['setosa', 'versicolor', 'virginica'] # 需要在这里也定义

# 创建 FastAPI 应用
app = FastAPI(title="Iris Classifier API via FastAPI", version="1.0")

# 全局加载模型
model = None
if os.path.exists(MODEL_FILENAME):
    model = joblib.load(MODEL_FILENAME)
    print(f"FastAPI: Model loaded from {MODEL_FILENAME}")
else:
    print(f"FastAPI Error: Model file '{MODEL_FILENAME}' not found.")

# 定义输入数据模型
class IrisInput(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float
    class Config:
        schema_extra = {"example": {"sepal_length": 5.1, "sepal_width": 3.5, "petal_length": 1.4, "petal_width": 0.2}}

# 定义输出数据模型
class PredictionOutput(BaseModel):
    prediction_index: int
    prediction_name: str

# 定义根路径
@app.get("/")
async def root():
    return {"message": "Iris Classifier API is running. Go to /docs for documentation."}

# 定义预测路径
@app.post("/predict", response_model=PredictionOutput)
async def predict(features: IrisInput):
    if model is None:
        raise HTTPException(status_code=503, detail="Model not loaded")
    
    try:
        input_array = np.array([[
            features.sepal_length, features.sepal_width,
            features.petal_length, features.petal_width
        ]])
        pred_index = model.predict(input_array)[0]
        pred_name = IRIS_TARGET_NAMES[pred_index] if 0 <= pred_index < len(IRIS_TARGET_NAMES) else "Unknown"
        
        return {"prediction_index": int(pred_index), "prediction_name": pred_name}
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Prediction error: {e}")

# --- End of main_fastapi_deploy.py ---

### FastAPI: 如何运行

1.  确保你已经将上面的代码保存到了 `main_fastapi_deploy.py` 文件中 (使用 `%%writefile`)。
2.  确保 `simple_iris_model_deploy.joblib` 模型文件在同一目录下。
3.  打开**终端**。
4.  运行命令： `uvicorn main_fastapi_deploy:app --reload`
5.  访问 `http://127.0.0.1:8000` 查看根信息。
6.  访问 `http://127.0.0.1:8000/docs` 查看并测试 API。

## 3. 使用 Gradio 创建交互式 Demo

Gradio 可以让你用几行代码为模型函数创建 Web UI。

In [ ]:
# --- Gradio: 导入与设置 ---
print("\n--- Setting up for Gradio ---")
import gradio as gr
import joblib
import numpy as np
import os

# --- Gradio: 加载模型与目标名称 (独立加载) ---
model_filename_gradio = 'simple_iris_model_deploy.joblib'
iris_target_names_gradio = ['setosa', 'versicolor', 'virginica']
model_gradio = None
if os.path.exists(model_filename_gradio):
    model_gradio = joblib.load(model_filename_gradio)
    print(f"Gradio: Model loaded from {model_filename_gradio}")
else:
    print(f"Gradio Error: Model file '{model_filename_gradio}' not found.")

# --- Gradio: 定义预测函数 ---
def predict_iris_gradio(sl, sw, pl, pw):
    if model_gradio is None: return "Error: Model not available."
    try:
        input_data = np.array([[sl, sw, pl, pw]])
        pred_index = model_gradio.predict(input_data)[0]
        pred_name = iris_target_names_gradio[pred_index] if 0 <= pred_index < len(iris_target_names_gradio) else "Unknown"
        
        # Gradio 的 Label 输出通常期望一个字典 {label: confidence}
        # 或者只返回最可能的标签字符串
        # 为了简单起见，我们只返回名字
        return pred_name
    except Exception as e:
        return f"Error: {e}"

# --- Gradio: 创建并启动 Interface --- 
if model_gradio:
    iface_gradio = gr.Interface(
        fn=predict_iris_gradio,
        inputs=[
            gr.Number(label="Sepal Length (cm)"),
            gr.Number(label="Sepal Width (cm)"),
            gr.Number(label="Petal Length (cm)"),
            gr.Number(label="Petal Width (cm)")
        ],
        outputs=gr.Textbox(label="Predicted Species"), # 使用 Textbox 显示结果
        title="Gradio Iris Predictor",
        description="Enter flower measurements to predict the Iris species using Gradio.",
        examples=[[5.1, 3.5, 1.4, 0.2], [6.7, 3.1, 4.7, 1.5], [7.3, 2.9, 6.3, 1.8]]
    )
    
    # 启动界面 (在 Jupyter 中会嵌入输出)
    # iface_gradio.launch() # 取消注释以启动
    print("\nGradio interface defined. Uncomment 'iface_gradio.launch()' to run it below.")
else:
    print("\nGradio: Skipping interface creation as model failed to load.")

### Gradio: 如何运行

1.  运行上面的代码单元格。
2.  取消最后一行 `iface_gradio.launch()` 的注释。
3.  再次运行该单元格。你应该会在单元格的输出区域看到一个可交互的 Web 界面。

## 4. 使用 Streamlit 构建数据应用

Streamlit 允许你使用 Python 脚本创建 Web 应用。每次与界面交互时，脚本会从头到尾重新运行。

**注意**: Streamlit 应用需要通过 `streamlit run <filename>.py` 命令从终端启动。

In [ ]:
%%writefile app_streamlit_deploy.py
# --- Streamlit: 文件内容 ---
import streamlit as st
import numpy as np
import joblib
import pandas as pd
from sklearn.datasets import load_iris
import os

MODEL_FILENAME_ST = 'simple_iris_model_deploy.joblib'

st.set_page_config(layout="wide") # Use wider layout

st.title("Iris Species Predictor (Streamlit)")
st.markdown("Use the sliders in the sidebar to input flower features and see the prediction.")

# --- 加载模型 (带缓存) ---
@st.cache_resource
def load_model_st():
    if os.path.exists(MODEL_FILENAME_ST):
        model = joblib.load(MODEL_FILENAME_ST)
        iris_names = load_iris().target_names
        print("Streamlit App: Model loaded.") # Prints in the terminal where streamlit runs
        return model, iris_names
    else:
        st.error(f"Model file '{MODEL_FILENAME_ST}' not found. Please ensure it's in the same directory.")
        return None, []

model_st, iris_target_names_st = load_model_st()

# --- 侧边栏输入 ---
st.sidebar.header("Input Features")
sl = st.sidebar.slider("Sepal Length (cm)", 4.0, 8.0, 5.1, 0.1)
sw = st.sidebar.slider("Sepal Width (cm)", 2.0, 4.5, 3.5, 0.1)
pl = st.sidebar.slider("Petal Length (cm)", 1.0, 7.0, 1.4, 0.1)
pw = st.sidebar.slider("Petal Width (cm)", 0.1, 2.5, 0.2, 0.1)

# --- 主区域显示 --- 
col1, col2 = st.columns(2)

with col1:
    st.subheader("Current Input Values")
    input_features = pd.DataFrame([[sl, sw, pl, pw]], 
                                columns=["Sepal Length", "Sepal Width", "Petal Length", "Petal Width"])
    st.dataframe(input_features)

with col2:
    st.subheader("Prediction")
    if model_st is not None:
        if st.button("Predict", key="predict_button"):
            input_array = input_features.values
            pred_index = model_st.predict(input_array)[0]
            pred_proba = model_st.predict_proba(input_array)[0]
            pred_name = iris_target_names_st[pred_index] if 0 <= pred_index < len(iris_target_names_st) else "Unknown"
            
            st.success(f"Predicted Species: **{pred_name}**")
            
            prob_df = pd.DataFrame({
                'Species': iris_target_names_st,
                'Probability': pred_proba
            })
            st.write("Probabilities:")
            st.dataframe(prob_df.style.format({'Probability': '{:.1%}'.format}))
        else:
            st.info("Click the 'Predict' button to see the result.")
    else:
        st.error("Model not loaded. Prediction unavailable.")

# --- 可选：显示数据样本 ---
st.sidebar.markdown("---")
if st.sidebar.checkbox("Show Raw Iris Data Sample"):
    st.subheader("Iris Dataset Sample (First 5 rows)")
    iris_df_st = pd.DataFrame(load_iris().data, columns=load_iris().feature_names)
    st.dataframe(iris_df_st.head())

# --- End of app_streamlit_deploy.py ---

### Streamlit: 如何运行

1.  确保你已经将上面的代码保存到了 `app_streamlit_deploy.py` 文件中。
2.  确保 `simple_iris_model_deploy.joblib` 模型文件在同一目录下。
3.  打开**终端**。
4.  运行命令： `streamlit run app_streamlit_deploy.py`
5.  Streamlit 应用会自动在你的浏览器中打开。

# 比较与总结

## 5. 比较与选择

| 特性              | FastAPI                        | Gradio                           | Streamlit                       |
|-------------------|--------------------------------|----------------------------------|---------------------------------|
| **主要用途**      | 构建高性能 API                  | 快速创建 ML 模型 Demo UI         | 构建交互式数据应用/Demo         |
| **编程模型**      | 标准 Web 框架 (路径操作函数)    | 定义函数 + 接口 (`Interface`)    | 脚本从上到下运行, Widget 触发重跑 |
| **UI 自定义**     | 低 (需要前端框架配合)           | 中等 (组件化, 主题)             | 中高 (布局, 组件, 自定义组件)   |
| **开发速度 (简单Demo)** | 中                             | 非常快                           | 很快                           |
| **部署**          | 生产级 API (配合 Uvicorn/Gunicorn) | Demo 共享 (临时链接), 可嵌入   | 应用部署 (Streamlit Cloud, Docker等) |
| **学习曲线**      | 中 (需要理解 Web API 概念)     | 低                               | 低                             |
| **适合场景**      | 模型作为服务 (MaaS), 后端 API | 快速验证模型, 分享 Demo        | 数据探索看板, 内部工具, ML Demo |

**选择建议**: 
*   如果你的主要目标是提供一个**稳健、高性能的 API** 供其他服务调用，选择 **FastAPI**。
*   如果你想**快速为模型创建一个简单的交互式 Demo**，特别是给非技术用户展示，**Gradio** 非常方便。
*   如果你想构建一个**交互式的数据应用、仪表板，或者稍微复杂一些的模型 Demo**，并且希望使用纯 Python 编写，**Streamlit** 是一个优秀的选择。

这三个工具各有优势，有时甚至可以结合使用（例如，用 FastAPI 构建后端 API，用 Streamlit 或 Gradio 构建前端用户界面来调用这个 API）。

## 总结

FastAPI, Gradio, 和 Streamlit 都是将机器学习模型从代码转化为实际应用或服务的强大工具。

*   **FastAPI** 专注于 API 构建。
*   **Gradio** 专注于快速 ML Demo UI。
*   **Streamlit** 专注于快速数据应用和交互式工具。

掌握这些工具可以让你更有效地展示、分享和部署你的机器学习成果。

In [ ]:
# --- Cleanup the saved model file --- 
if os.path.exists(model_filename):
    os.remove(model_filename)
    print(f"Cleaned up model file: {model_filename}")
# Clean up files possibly created by %%writefile 
if os.path.exists("main_fastapi_deploy.py"):
    os.remove("main_fastapi_deploy.py")
if os.path.exists("app_streamlit_deploy.py"):
     os.remove("app_streamlit_deploy.py")